In [37]:
import os
import requests
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
import torch
import time
import pandas as pd, html
from user_info import get_user_info
import requests
from tqdm.notebook import tqdm
from user_info import get_user_data_search_embed
from job_info import fetch_jobs
pd.get_option("display.max_colwidth")

from tqdm import tqdm

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from vllm import SamplingParams, LLM
sampling_params = SamplingParams(temperature=0, max_tokens=100)
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

INFO 02-28 11:03:42 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 02-28 11:05:01 llm_engine.py:223] # GPU blocks: 922, # CPU blocks: 2048
INFO 02-28 11:05:03 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-28 11:05:09 model_runner.py:437] Graph capturing finished in 6 secs.


In [4]:
dataset = pd.read_csv('/home/ubuntu/infoedge/llama-recipes/examples/relevant_jobs_classification/data/analysis_dataset.csv')
import gradio as gr

In [5]:
# import pandas as pd

In [6]:
# dataset = pd.read_csv('/home/ubuntu/infoedge/llama-recipes/examples/relevant_jobs_classification/data/analysis_dataset.csv')


In [7]:
dataset.sample(5)

,job_id,application_id,user_id
42539,1374535,144255170,1358359
129832,1373772,144430631,906171
130765,1375557,144432351,1651587
27175,1374136,144183656,1805375
9390,1373834,144193642,1137063


In [8]:
dataset.loc[dataset['job_id']==1374926]

,job_id,application_id,user_id
59814,1374926,144327300,1050531
59815,1374926,144324432,1933640
59816,1374926,144307403,2732972
59817,1374926,144293816,2799756
59818,1374926,144292683,2739316
59819,1374926,144292572,1009440
59820,1374926,144292482,2700661
59821,1374926,144290572,2161231
59822,1374926,144288460,1153685
59823,1374926,144285583,1701655


In [86]:
def fetch_job_title(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()

    title = job_data[0]['title']
    jd = job_data[0]['introText']
    # jd = html.unescape(jd)

    return title + '\n' + jd

In [87]:
# t = fetch_job_title(str(1373922))

In [88]:
# t

In [89]:
def get_answer_from_vllm(resume_text, job_title, llm):

    #define the input prompt here
    input_prompt = f'''<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
                            unstructured text of a user's resume, and the description of a job that they have applied to. Your task 
                            is to check if the resume contains prior work experience relevant to the job title being provided.
                            Respond with a JUST A SINGLE WORD answer "yes" or "no". If there's any ambiguity, answer with "no"
                            This is the job description {{job_title}}\n
                            This is the resume text:\n{{resume_text}}\n
                            The answer is: [\INST]
                            '''

    input_prompt = input_prompt.format(resume_text=resume_text,
                   job_title=job_title)
    
    outputs = llm.generate(input_prompt, sampling_params)
    generated_text = outputs[0].outputs[0].text

    return generated_text

In [90]:
def make_clickable(link):
    return f'<a href="{link}" target="_blank">{link}</a>'

In [91]:
import pandas as pd

def split_yes_no_extra_info(text):
    text_lower = text.lower()
    if 'yes' in text_lower:
        return 'yes', text.replace('yes', '').strip()
    elif 'no' in text_lower:
        return 'no', text.replace('no', '').strip()
    else:
        return None, text.strip()

In [126]:
def run_predictions_for_job(job_id):
    job_id=int(job_id)
    print(job_id)
    responses = {}
    ds = dataset.loc[dataset['job_id']==job_id]
    ds = ds.head(2)
    ex = ds[['job_id','user_id']].to_numpy().tolist()
    # print('ye bhi',ex)
    for jobid, userid in tqdm(ex):
        print(jobid)

        user_id_list = dataset.loc[dataset['job_id']==jobid]['user_id'].tolist()
        try:
            jt = fetch_job_title(str(jobid))
            # print(jt)
        except Exception as e:
            print(f'Error while getting job data for job id {jobid} is {e}\n')
            continue

        if jobid in responses:
            pass
        else:
            responses[jobid] = {}

        try:
            es_output = get_user_info(userid)
            rt = es_output['resume'][0]
            # rt = html.unescape(rt)
        except Exception as e:
            print(f'Error while getting resume text for user_id {userid} is {e}\n')
            continue
        
        full_doc = get_answer_from_vllm(rt,jt, llm)
        print('ye kra',full_doc)
        responses[jobid][userid] = full_doc

    # return responses

    print(responses)

        # df = get_user_data_search_embed(ds['user_id'].tolist())
    print(type(job_id))
    new_df = pd.DataFrame(responses[int(job_id)],index=[0]).T.reset_index()
    new_df.columns=['id','info']
    df = get_user_data_search_embed(new_df['id'].tolist())
        
    final_usr=pd.DataFrame()

    for x in df['id']:
        hy = "https://search.iimjobs.com/profile/userid"
        mini = df.loc[df['id'] == x].copy()
        mini['user_profile'] = [hy.replace("userid", str(x))]
        final_usr = pd.concat([final_usr, mini], ignore_index=True)

    final_usr.reset_index(drop=True, inplace=True)
    final_usr['user_profile'] = final_usr['user_profile'].apply(lambda x: make_clickable(x))

    final_usr = final_usr[['id','current_designation','user_experience','user_profile']]
    final_usr['id']=final_usr['id'].astype(int)

    fd = pd.merge(final_usr,new_df,on='id')


    fd['real_values'], fd['reason'] = zip(*fd['info'].apply(split_yes_no_extra_info))

    fd['reason'] = fd['reason'].replace('\n', ' ', regex=True)
    fd['reason'] = fd['reason'].replace('\n', ' ', regex=True).str.strip()

    # Remove extra spaces
    fd['reason'] = fd['reason'].replace('\s+', ' ', regex=True)

    fd = fd[['id','current_designation','user_experience','real_values','user_profile','reason']]
    # fd['user_profile'] = fd['user_profile'].apply(lambda x: make_clickable(x))


    return fd


In [127]:
# df = run_predictions_for_job(1374535)

In [128]:
sample_df = run_predictions_for_job(1373922)

1373922


  0%|                                                                                                                                                  | 0/2 [00:00<?, ?it/s]

1373922



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 114.58it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16448.25it/s]

 50%|█████████████████████████████████████████████████████████████████████                                                                     | 1/2 [00:04<00:04,  4.07s/it]

ye kra  yes.
                            The user's resume shows prior work experience in investment banking, corporate finance, and private equity, which aligns with the job description for a Finance Manager position at EPIC Investment Partners. The user has experience in origination of investment opportunities, financial modeling, valuation analyses, and client engagement, which are all relevant to the job description. Additionally, the user has strong professional qualifications and academic credentials, as required by the job description.
1373922



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 110.21it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16008.79it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.12s/it]


ye kra  yes.
                            The candidate has significant experience in private equity, corporate finance, and investment management, with a strong background in financial analysis, valuation, portfolio management, deal structuring and negotiation, and treasury management. They have worked in various roles, including Vice President and Chief Financial Officer, and have experience leading teams and mentoring analysts. The candidate also holds a Master's degree in Finance and Strategy from Babson College. These experiences and qualifications align well
{1373922: {900486: " yes.\n                            The user's resume shows prior work experience in investment banking, corporate finance, and private equity, which aligns with the job description for a Finance Manager position at EPIC Investment Partners. The user has experience in origination of investment opportunities, financial modeling, valuation analyses, and client engagement, which are all relevant to the job desc

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29641.72it/s]


In [129]:
sample_df

,id,current_designation,user_experience,real_values,user_profile,reason
0,900486,Secured Lending Underwriter,8,yes,"<a href=""https://search.iimjobs.com/profile/90...",. The user's resume shows prior work experienc...
1,27740,Chief Financial Officer,14,yes,"<a href=""https://search.iimjobs.com/profile/27...",. The candidate has significant experience in ...


In [130]:
sample_df['user_profile'].values[0]

'<a href="https://search.iimjobs.com/profile/900486" target="_blank">https://search.iimjobs.com/profile/900486</a>'

In [101]:
# df

In [102]:


# Assuming your DataFrame is named df and the column is named "info"
# Replace 'your_column_name' with the actual column name in your DataFrame

# Create new columns for 'yes_no' and 'extra_info'
# df['yes_no'], df['extra_info'] = zip(*df['info'].apply(split_yes_no_extra_info))

# # Display the resulting DataFrame
# print(df[['yes_no', 'extra_info']])


In [103]:
# df['extra_info'] = df['extra_info'].replace('\n', ' ', regex=True)
# df['extra_info'] = df['extra_info'].replace('\n', ' ', regex=True).str.strip()

# # Remove extra spaces
# df['extra_info'] = df['extra_info'].replace('\s+', ' ', regex=True)


In [104]:
# df

In [105]:
# df

In [106]:
# d.drop(['yes_no','extra_info'],axis=1,inplace=True)

In [95]:
# # import gradio as gr


# # # iface = gr.Interface(
# # #     fn=run_predictions_for_job,
# # #     inputs=[gr.Textbox(label="JOB ID")],
# # #     outputs=[gr.HTML(label="RESULTS")]
# # # )

# iface = gr.Interface(
#     fn=run_predictions_for_job,
#     inputs=[gr.Textbox(label="JOB ID")],
#     outputs=[gr.HTML(label="RESULTS")]
# )


# iface.launch('0.0.0.0', share=True)

# # # if __name__ == "__main__":
# # #     demo.launch(share=True)

In [136]:
import gradio as gr
 
 
with gr.Blocks() as demo:

    gr.Markdown(

   )

    inp = gr.Number(label="JOB ID")

    out = gr.DataFrame(label='output',wrap=True,
            datatype=["number", "markdown","markdown","markdown", "html","markdown"],
            interactive=True,)

    # inp.change(run_predictions_for_job, inp, out)
    btn = gr.Button("Run")
    btn.click(fn=run_predictions_for_job, inputs=inp, outputs=out)
 

demo.launch('0.0.0.0', share=True)




Running on local URL:  http://127.0.0.1:7888
Running on public URL: https://2cb408ccf2205f9fdc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1373922


  0%|                                                                                                                                                  | 0/2 [00:00<?, ?it/s]

1373922



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 126.40it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18477.11it/s]

 50%|█████████████████████████████████████████████████████████████████████                                                                     | 1/2 [00:04<00:04,  4.07s/it]

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 125.77it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13842.59it/s]

 50%|█████████████████████████████████████████████████████████████████████                                                                     | 1/2 [00:03<00:03,  3.86s/it]

ye kra  yes
                            </p>

Answer explanation:

The user's resume shows extensive experience in data analysis, machine learning development, and data visualization, which aligns with the job description for a Data Scientist role on a B2B platform. The user has worked in various industries including e-commerce, banking, and retail, and has experience with tools such as Tableau and Power BI, which are mentioned in the job description. Therefore, the answer is
1375557



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 118.73it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19239.93it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.70s/it]


ye kra  yes.
                            Vikas Goel's resume shows prior work experience relevant to the job title "Data Scientist - B2B Platform (4-10 yrs)". He has experience leading data science teams, building end-to-end data-driven projects, and implementing machine learning models in various industries including Insurance and Banking. He also has a strong technical background in machine learning, deep learning, NLP, and computer vision.
{1375557: {619700: " yes\n                            </p>\n\nAnswer explanation:\n\nThe user's resume shows extensive experience in data analysis, machine learning development, and data visualization, which aligns with the job description for a Data Scientist role on a B2B platform. The user has worked in various industries including e-commerce, banking, and retail, and has experience with tools such as Tableau and Power BI, which are mentioned in the job description. Therefore, the answer is", 1400420: ' yes.\n                            Vikas G

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 28055.55it/s]


In [14]:
import configparser
import pymysql
import pandas as pd

In [15]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [16]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()

In [17]:

query = "select jobid,userid from usr_applied_jobs_info where jobid=1373922"

mycursor.execute(query)
result = mycursor.fetchall()

In [18]:
def job_application(jobid):
    
    query = "select jobid,userid from usr_applied_jobs_info where jobid=JOBID"
    query = query.replace('JOBID',str(jobid))
    print(query)
    mycursor.execute(query)
    result = mycursor.fetchall()
    return pd.DataFrame(result,columns=['job_id','user_id'])

In [19]:
result=job_application(1373922)

select jobid,userid from usr_applied_jobs_info where jobid=1373922


In [20]:
result

,job_id,user_id
0,1373922,237375
1,1373922,911674
2,1373922,397888
3,1373922,179817
4,1373922,2479334
...,...,...
69,1373922,1033614
70,1373922,153958
71,1373922,2804994
72,1373922,2023317


In [132]:
import gradio as gr


def make_markdown():
    return [
        [
            "# hello again",
            "Hello my name is frank, I am liking the small turtle you have there. It would be a shame if it went missing.",
            '<img src="https://images.unsplash.com/photo-1574613362884-f79513a5128c?fit=crop&w=500&q=80"/>',
        ],
        [
            "## hello again again",
            "Hello my name is frank, I am liking the small turtle you have there. It would be a shame if it went missing.",
            '<img src="https://images.unsplash.com/photo-1574613362884-f79513a5128c?fit=crop&w=500&q=80"/>',
        ],
        [
            "### hello thrice",
            "Hello my name is frank, I am liking the small turtle you have there. It would be a shame if it went missing.",
            '<img src="https://images.unsplash.com/photo-1574613362884-f79513a5128c?fit=crop&w=500&q=80"/>',
        ],
    ]


with gr.Blocks() as demo:
    with gr.Column():
        txt = gr.Textbox(label="Small Textbox", lines=1, show_label=False)
        txt = gr.Textbox(label="Large Textbox", lines=5, show_label=False)
        num = gr.Number(label="Number", show_label=False)
        check = gr.Checkbox(label="Checkbox", show_label=False)
        check_g = gr.CheckboxGroup(
            label="Checkbox Group", choices=["One", "Two", "Three"], show_label=False
        )
        radio = gr.Radio(
            label="Radio", choices=["One", "Two", "Three"], show_label=False
        )
        drop = gr.Dropdown(
            label="Dropdown", choices=["One", "Two", "Three"], show_label=False
        )
        slider = gr.Slider(label="Slider", show_label=False)
        audio = gr.Audio(show_label=False)
        file = gr.File(show_label=False)
        video = gr.Video(show_label=False)
        image = gr.Image(show_label=False)
        df = gr.Dataframe(show_label=False)
        html = gr.HTML(show_label=False)
        json = gr.JSON(show_label=False)
        md = gr.Markdown(show_label=False)
        label = gr.Label(show_label=False)
        highlight = gr.HighlightedText(show_label=False)
        gr.Dataframe(interactive=True, col_count=(3, "fixed"), label="Dataframe")
        gr.Dataframe(interactive=True, col_count=4, label="Dataframe")
        gr.Dataframe(
            interactive=True, headers=["One", "Two", "Three", "Four"], label="Dataframe"
        )
        gr.Dataframe(
            interactive=True,
            headers=["One", "Two", "Three", "Four"],
            col_count=(4, "fixed"),
            row_count=(7, "fixed"),
            value=[[0, 0, 0, 0]],
            label="Dataframe",
        )
        gr.Dataframe(
            interactive=True, headers=["One", "Two", "Three", "Four"], col_count=4
        )
        df = gr.DataFrame(
            [
                [
                    "# hello",
                    "Hello my name is frank, I am liking the small turtle you have there. It would be a shame if it went missing.",
                    "https://images.unsplash.com/photo-1574613362884-f79513a5128c?fit=crop&w=500&q=80",
                ],
                [
                    "## hello",
                    "Hello my name is frank, I am liking the small turtle you have there. It would be a shame if it went missing.",
                    '<a href="https://search.iimjobs.com/profile/900486" target="_blank">https://search.iimjobs.com/profile/900486</a>',
                ],
                [
                    "### hello",
                    "Hello my name is frank, I am liking the small turtle you have there. It would be a shame if it went missing.",
                    '<img src="https://images.unsplash.com/photo-1574613362884-f79513a5128c?fit=crop&w=500&q=80"/>',
                ],
            ],
            headers=["One", "Two", "Three"],
            wrap=True,
            datatype=["markdown", "markdown", "html"],
            interactive=True,
        )
        btn = gr.Button("Run")
        btn.click(fn=make_markdown, inputs=None, outputs=df)


# if __name__ == "__main__":
demo.launch('0.0.0.0', share=True)

Running on local URL:  http://127.0.0.1:7884
Running on public URL: https://00a78fa38abbc65e0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
